<a href="https://colab.research.google.com/github/sai974/RA_stuff/blob/main/Digit_Triplet_Test_Mishra_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!pip install pydub

Mounted at /content/drive/


In [ ]:
import random
import IPython.display as ipd
import time
import pygame
from pydub import AudioSegment
import soundfile
from scipy.io import wavfile
import numpy as np

pygame 2.5.1 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


Read Data from the google drive

In [ ]:
digit_base_path = '/content/drive/MyDrive/DigitTripletTest/Digits/'
digit_file_path_arr = []
num_base_files = 10
for i in range(num_base_files):
  digit_file_path_arr.insert(i, digit_base_path + str(i) + '.wav')

print(digit_file_path_arr)

background_base_path = '/content/drive/MyDrive/DigitTripletTest/SpeechShapedNoiseShort.wav' # check dimensions, monochannel or stereo audio - dimensions probably don't match, crop background noise

['/content/drive/MyDrive/DigitTripletTest/Digits/0.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/1.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/2.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/3.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/4.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/5.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/6.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/7.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/8.wav', '/content/drive/MyDrive/DigitTripletTest/Digits/9.wav']


In [91]:
import numpy as np

def generate_triplet(input_set, N):
    rand_index_int = list(np.random.randint(low=0, high=N, size=(N)))
    input_set_out = [input_set[i] for i in rand_index_int]
    return tuple(input_set_out)

def check_diff(input1:tuple, input2:tuple):
  equal_ind = np.not_equal(input1, input2)
  sum_equal = np.sum(equal_ind)
  return (sum_equal > 2)

In [120]:
def scale_signal_to_dB(signal, dB):
  rms_65dB =  10.0**(dB/20.0);
  RMS_sig = np.sqrt(np.mean(np.power(signal, 2)))
  signal_scale_factor = rms_65dB/RMS_sig

  scaled_signal = np.multiply(signal, signal_scale_factor) # the signal is now 65dB

  return scaled_signal

In [93]:
input_set = range(10)
N = 3
rand_index_int = list(np.random.randint(low=0, high=N, size=(N)))
input_set_out = [input_set[i] for i in rand_index_int]
input_set_out = tuple(input_set_out)
print(input_set_out)

check_diff(input_set_out, input_set_out)

(2, 2, 1)


False

In [96]:
def generate_digit_Trials(check_diff, num_trials, num_digits, x_arr):
  trial_arr = []
  for ii in range(num_trials):
      if (ii == 0):
          trial_arr.insert(ii, generate_triplet(x_arr, num_digits))
          continue
      else:
         out = generate_triplet(x_arr, num_digits)
         while (check_diff(out, trial_arr[ii - 1])):
          out = generate_triplet(x_arr, num_digits)
         trial_arr.insert(ii, out)

  return trial_arr

In [ ]:
def get_noisy_signal(signal, noise, signal_dB, req_SNR_dB):
  signal = np.array(signal)
  noise = np.array(noise)

  scaled_signal = scale_signal_to_dB(signal, signal_dB)

  noise = noise[0:len(signal)]
  noise_dB = signal_dB - req_SNR_dB

  scaled_noise = scale_signal_to_dB(noise, noise_dB)

  noisy_signal = signal + noise

  return noisy_signal

In [126]:
def create_triplet_audio(triplet, num_digits, SNR_dB, audio_file_arr, background_file_arr):
  combined_signal  = np.array([])
  for i in triplet:
    sample_rate, signal = wavfile.read(audio_file_arr[i]) # Read Signal
    combined_signal = np.append(combined_signal, signal)

  noise_sample_rate, noise = wavfile.read(background_file_arr)
  noise = noise[0: len(combined_signal)]
  noisy_signal  = get_noisy_signal(combined_signal, noise, 65, 60)

  soundfile.write('tmp.wav', noisy_signal, sample_rate)




In [135]:
x_arr = range(10)
trial_digits = generate_digit_Trials(check_diff, 2, 3, x_arr)
print(trial_digits)

for i in trial_digits:
  create_triplet_audio(i, 3, 1000, digit_file_path_arr, background_base_path)

ipd.display(ipd.Audio('tmp.wav', autoplay=True))


[(0, 2, 1), (0, 2, 1)]


<ipython-input-126-3df6094c1b98>:4: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, signal = wavfile.read(audio_file_arr[i]) # Read Signal
<ipython-input-120-db555d030750>:3: RuntimeWarning: invalid value encountered in sqrt
  RMS_sig = np.sqrt(np.mean(np.power(signal, 2)))


In [ ]:
# from scipy.io import wavfile
# import matplotlib.pyplot as plt
# import numpy as np

# # Read sample .wav file
# samplerate, data = wavfile.read(zero)
# # Add noise to the signal
# samplerate_noise, noise = wavfile.read(background)
# length = len(data)
# time = np.linspace(0., length, data.shape[0])
# # Plotting a simple sound file
# plt.plot(time, data, label="Zero")
# plt.legend()
# plt.xlabel("Time [s]")
# plt.ylabel("Amplitude")
# plt.show()
# # Plotting the sound file with noise
# data_with_noise = data + noise[0:len(data)]
# plt.plot(time, data_with_noise, label="Zero")
# plt.legend()
# plt.xlabel("Time [s]")
# plt.ylabel("Amplitude")
# plt.show()
# # Plotting the Spectrogram
# from scipy import signal
# f, t, Sxx = signal.spectrogram(data_with_noise, samplerate)
# plt.pcolormesh(t, f, Sxx, shading='gouraud')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

In [ ]:
# ADD: 1000 Hz noise to signify the start of the test

# output the audio of the three numbers with background noise
i = 0
while i < 3:
    num = triplet[i]
    samplerate, signal = wavfile.read(audio[num]) # Read Signal
    samplerate_noise, noise = wavfile.read(background) # Read Noise
    length_signal = len(signal) # Signal Length
    signal_noise = signal + noise[0:length_signal] # Add Noise
    soundfile.write('tmp.wav',signal_noise,samplerate)
    sound = AudioSegment.from_wav('tmp.wav')
    ipd.display(ipd.Audio('tmp.wav', autoplay=True))
    time.sleep(sound.duration_seconds + 0.30)  # Add a 300 ms buffer after playback
    i = i + 1


# plotting frequency of audio file (use tfio)
# import matplotlib.pyplot as plt
# from scipy import signal
# from scipy.io import wavfile
# import numpy as np

# sample_rate, samples = wavfile.read(audio[1])
# frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

#get maximum
# x,y=np.where(spectrogram == spectrogram.max())
# print("Frequency index where the maximum is")
# print(x)
# print("Frequency Value")
# print(frequencies[x])

<ipython-input-5-67d14d244833>:7: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, signal = wavfile.read(audio[num]) # Read Signal
